In [ ]:
import pathlib
import json
from helpers import ai
from pprint import pprint

In [ ]:
NBS_DIR = pathlib.Path().resolve().parent
BASE_DIR = NBS_DIR
DATASET_DIR = BASE_DIR / "dataset"

In [ ]:
for path in list(DATASET_DIR.glob("**/**/detail.json")):
    post_detail_dir = path.parent
    post_detail = json.loads(path.read_text())
    podcasts_dir = post_detail_dir / "podcasts"
    recommended_podcasts_dir = post_detail_dir / "recommended"
    for transcript_path in podcasts_dir.glob("**/**/transcript.txt"):
        podcast_detail_dir = transcript_path.parent
        podcast_detail_path = podcast_detail_dir / 'podcast.json'
        if not podcast_detail_path.exists():
            continue
        podcast_detail = json.loads(podcast_detail_path.read_text())
        podcast_title = podcast_detail.get('trackName')
        podcast_id = podcast_detail.get('trackId')
        podcast_description = podcast_detail.get('description')
        apple_podcast_track_url = podcast_detail.get('trackViewUrl')
        episode_url = podcast_detail.get('episodeUrl')
        content = transcript_path.read_text()
        try:
            result_data, is_json = ai.extract_summary_and_keywords(content=content)
        except:
            result_data = {'summary': None, 'keywords': []}
        recommendation = {
            'title': podcast_title,
            'description': podcast_description,
            'links': {
                'applePodcasts': apple_podcast_track_url,
                'direct': episode_url,
            },
            'ai_summary': result_data.get('summary'),
            'ai_keywords': result_data.get('keywords'),
        }
        print(recommendation)
        recommended_podcasts_dir.mkdir(exist_ok=True)
        outpath = recommended_podcasts_dir / f"{podcast_id}.json"
        outpath.write_text(json.dumps(recommendation))

In [ ]:
for path in list(DATASET_DIR.glob("**/recommended/**")):
    print(path)
    for rec_path in path.glob("*.json"):
        pprint(json.loads(rec_path.read_text()))